In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torchtext

torchtext.disable_torchtext_deprecation_warning()

np.random.seed(42)

DEVICE = (
    torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
)
print(f"Training on {DEVICE} using PyTorch {torch.__version__}")
# torch.set_default_device(DEVICE)

Training on mps using PyTorch 2.3.1


In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

from dataset import load_dataset, to_dataloader


# Load the dataset
df, vocab, label_encoder = load_dataset()

# Split the data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["category"], test_size=0.2, random_state=42
)

train_loader = to_dataloader(train_texts, train_labels, vocab)
test_loader = to_dataloader(test_texts, test_labels, vocab, shuffle=False)

/Users/gabriel/federated-text-classification/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package reuters to /Users/gabriel/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabriel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from text_rnn import TextRNN

# Model hyperparameters
vocab_size = len(vocab)
output_size = len(label_encoder.classes_)

model = TextRNN(
    vocab_size,
    output_size,
    padding_idx=vocab["<pad>"],
).to(DEVICE)

In [5]:
import torch.optim as optim
from torch import nn


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    num_epochs: int,
    verbose=False,
):
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    model.train()

    for epoch in range(num_epochs):
        for texts, labels in train_loader:
            texts, labels = texts.to(DEVICE), labels.to(DEVICE)

            outputs = model(texts)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if verbose:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


train_model(model, train_loader, num_epochs=10, verbose=True)

Epoch [1/1], Loss: 1.0129


In [10]:
def evaluate_model(model: nn.Module, test_loader: DataLoader):
    model.eval()

    total, correct = 0, 0

    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(DEVICE), labels.to(DEVICE)

            outputs = model(texts)
            _, predicted = torch.max(outputs, 1)

            labels = labels
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    print(f"Accuracy: {accuracy * 100:.2f}%")


evaluate_model(model, test_loader)

Accuracy: 66.54%
